In [ ]:
import gdown
import pandas as pd 

In [4]:
gdown.download(
    'https://docs.google.com/spreadsheets/d/1YTd9Z-w9CrVEW4Mj_Qjv2G8ZEfxA91B8mcfQShM38SY/edit?usp=sharing',
    fuzzy=True,
    output='cache/type_classification_annotations.xlsx'
)

Downloading...
From (original): https://drive.google.com/uc?id=1YTd9Z-w9CrVEW4Mj_Qjv2G8ZEfxA91B8mcfQShM38SY
From (redirected): https://docs.google.com/spreadsheets/d/1YTd9Z-w9CrVEW4Mj_Qjv2G8ZEfxA91B8mcfQShM38SY/export?format=xlsx
To: /Users/spangher/Projects/berkeley-research/news-interview-question-generation/notebooks/cache/type_classification_annotations.xlsx
4.61MB [00:00, 11.4MB/s]


'cache/type_classification_annotations.xlsx'

In [7]:
annotated_df = pd.read_excel('cache/type_classification_annotations.xlsx', sheet_name='all_interviews_combined_v6')

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer

In [43]:
label_df = annotated_df[['Question type', 'Human Label']].apply(lambda s: s.str.lower()).dropna()
label_df = label_df.apply(lambda s: s.str.replace('-', ' ').str.replace('starting ', 'starting/ending ').str.split(r',\s', regex=True))

In [44]:
mlb= MultiLabelBinarizer()

In [45]:
mlb.fit(label_df.unstack())

MultiLabelBinarizer()

In [46]:
pd.Series(mlb.classes_).sort_values()

0       acknowledgement statement
1             broadening question
2              challenge question
3             definition question
4              follow up question
5    opinion/speculation question
6         starting/ending remarks
7       topic transition question
8           verification question
dtype: object

In [ ]:
y_pred_df = label_df['Question type'].pipe(lambda s: pd.DataFrame(mlb.transform(s), columns=mlb.classes_))
y_true_df = label_df['Human Label'].pipe(lambda s: pd.DataFrame(mlb.transform(s), columns=mlb.classes_))

In [61]:
from sklearn.metrics import f1_score, recall_score, precision_score

In [79]:
results_df = pd.concat([
    pd.Series(precision_score(y_true_df, y_pred_df, average=None)).to_frame('precision'),
    pd.Series(recall_score(y_true_df, y_pred_df, average=None)).to_frame('recall'),
    pd.Series(f1_score(y_true_df, y_pred_df, average=None)).to_frame('f1-score')
], axis=1)
results_df.index = mlb.classes_

In [80]:
results_df.round(2)

,precision,recall,f1-score
acknowledgement statement,1.00,1.00,1.00
broadening question,0.58,0.70,0.64
challenge question,1.00,1.00,1.00
definition question,1.00,1.00,1.00
follow up question,0.59,0.87,0.70
opinion/speculation question,0.75,0.75,0.75
starting/ending remarks,0.90,1.00,0.95
topic transition question,0.75,0.47,0.58
verification question,0.89,0.80,0.84
